**Authors:**  Viktor Domazetoski<br>
**Copyright:** 2022 Viktor Domazetoski<br>
**License:** Apache License 2.0

<div class="alert alert-block alert-success">
This notebook takes the WIOT 2016 release excels as input and calculates the Country-aggregated world-Input-Ouput Tables (CIOTs) as well as all other necessary vectors, matrices and tensors. 

# Imports

In [1]:
"""Numerical and statistical libraries"""
import numpy as np
import pandas as pd
from scipy import stats
from sklearn.preprocessing import StandardScaler

"""Other libraries"""
import pickle
import time

# CIOT Class

- $\mathbf{Z}$ matrix - uniquely determined by the 4-order tensor $\mathcal{Z} \in \mathbb{R}^{J\times J\times S\times S}$. The entries of $\mathcal{Z}$ are entries $z_{\hat{i} \hat{j}}^{rs}$ describing the intermediate purchases (input flows) by industry $s$ in country $\hat{j}$ from sector $r$ in country $\hat{i}$.


- $F$ vector - uniquely determined by 3-order tensor $\mathcal{F} \in \mathbb{R}^{J\times J\times S}$. The entries of $\mathcal{F}$ entries $f_{\hat{i} \hat{j}}^r$ denoting the final use in each country $\hat{j}$ of output originating from sector $r$ in country $\hat{i}$.
\begin{align}
f_{\hat{i}}^r &= \sum_{\hat{j} =1 }^J f_{ \hat{i} \hat{j}}^r
\end{align}

- $X$ vector - where $x_{\hat{i}}^r$ represents the value of gross output originating from sector $r$ in country $\hat{i}$
\begin{align}
x_{\hat{i}}^r &= \sum_{s=1}^S \sum_{\hat{j}=1}^J z_{\hat{i} \hat{j}}^{rs} + f_{\hat{i}}^r
\end{align}

- $B$ vector - designates the weight of the good $i$ (produced by country-industry pair $i$) in the representative household's preferences (with the normalization $\sum_i \beta_i = 1$).
\begin{align}
\beta_{i}^r &= \frac{f_{i}^r}{\sum_{i,r}f_{i}^r}
\end{align}

- $E$ vector - is the logarithm of the Hicks-neutral technology. $\varepsilon_i$ are microeconomic shocks that are i.i.d. across country-industry pairs, are symmetrically distributed around the origin with full support over $\mathbb{R}$, and have a finite standard deviation, which we normalize to one. 


- $\mathbf{A}^{(1)}$ matrix - adjacency matrix of the country-sector network.

\begin{align}
a_{i j}^{rs} =  \frac{ z_{ij}^{rs} }{ x_{j}^{s} } 
\end{align}

- $\mathbf{A}^{(2)}$ matrix - adjacency matrix of the aggregated country network.

\begin{align}
a_{i j} =  \frac{\sum_{r,s} z_{ij}^{rs} }{\sum_{s} x_{j}^{s}} 
\end{align}

- $\mathbf{A}^{(3)}$ matrix - adjacency matrix of the aggregated sector network.
\begin{align}
a_{rs} =  \frac{ \sum_{ i, j} z_{ i j }^{rs} }{ \sum_{j} x_{ j}^{s} }  
\end{align}

- $\Delta \mathbf{A}$ - Differential of A matrix


- $\mathbf{L}$ matrix - Leontief-inverse matrix
\begin{align}
L = (I-A)^{-1}
\end{align}


- $\frac{ d \mathbf{L} }{ d \mathbf{A}}$ - The derivative of the Leontief-inverse matrix $\mathbf{L}$  with respect to the matrix $ \mathbf{A}$ is an $n^2 \times n^2$ matrix.
\begin{align} 
\frac{ d \mathbf{L} }{ d \mathbf{A}} =  
\begin{bmatrix}
\mathbf{L}_{11} &  \mathbf{L}_{12} & \ldots &  \mathbf{L}_{in} \\  
\vdots & \vdots &  \ddots & \vdots \\
 \mathbf{L}_{n1} &  \mathbf{L}_{n2} & \ldots &  \mathbf{L}_{nn}
\end{bmatrix} = 
\mathbf{L}^T \otimes \mathbf{L}
\end{align}

- $\Delta \mathbf{L}$
\begin{align}
\Delta \mathbf{L} = \frac{ d \mathbf{L} }{ d \mathbf{A}} \Delta \mathbf{A}
\end{align}

\begin{align}
\frac{d \, \mbox{vec} \, \mathbf{L} }{ d \, \boldsymbol{\theta}^T } =  \left( \mathbf{L}^T \otimes \mathbf{L}    \right)   \frac{d \, \mbox{vec} \, \mathbf{A} }{ d \,  \boldsymbol{\theta}^T  } 
\end{align}

- $\frac{ d \mathbf{L} }{ d a_{ij}}$ - The derivative of the Leontief-inverse matrix $\mathbf{L}$  with respect to the scalar $a_{ij}$ is an $n \times n$ matrix:    
\begin{align}
\frac{ d \mathbf{L} }{ d a_{ij}}  =  \left[  \ell_{jp} \ell_{qi}      \right],     
\end{align}

- We then define a tensor $\mathcal{P} \in \mathbb{R}^{J\times J\times J\times J}$, which will be called $\textit{shock matrix}$, as follows:
\begin{align}
\mathcal{P} = \left[ \log p^{uv}_{ij} \right] = \left[\ell_{vi} \ell_{ju}  \Delta a^{uv} \beta_j \varepsilon_i  \right]
\end{align}

In [2]:
class CIOT:
    def __init__(self,
                 excel_dataframe=None,
                 year=None,
                 N_sectors=56,
                 N_countries=44):
        """Initialize Country Input Output class.
        
        Input: 
            excel_dataframe (pd.DataFrame): World Input Output Data, currently configured for data from 2016 WIOT release
            year (str): Year of the WIOT table. For the 2016 WIOT release it ranges from 2000 to 2014
            N_sectors (int): Number of sectors in the WIOT table. For the 2016 WIOT release it is 56
            N_countries (int): Number of countries in the WIOT table. For the 2016 WIOT release it is 44
        """

        self.excel_dataframe = excel_dataframe  # World Input Output Table Data
        self.year = year  # Year of the WIOT table
        self.N_sectors = N_sectors  # Number of sectors
        self.N_countries = N_countries  # Number of countries
        self.N = N_countries * N_sectors  # Number of sectors * countries in the WIOT table
        self.country_names = None  # Country names of the WIOT table

        self.z = None  # Z matrix
        self.x = None  # x vector
        self.f = None  # f vector

        self.beta = None  # beta vector
        self.eps = None  # eps vector

        self.A = None  # A matrix
        self.L = None  # L matrix

        self.delta_A = None  # delta A matrix
        self.dL = None  # dL matrix - derivative of L
        self.delta_L = None  # delta L matrix - deltaA * dL

        self.P = None  # shock matrix P

    def get_country_names(self):
        """Set and return country names from the Input Data"""
        country_names = []
        for i in range(self.N_countries):
            country_names.append(
                self.excel_dataframe.columns[i * self.N_sectors][2])
        self.country_names = country_names
        return country_names

    def calculate_country_Z_matrix(self):
        """Calculate the country-aggregated Z matrix from the Input Data"""
        Z = np.zeros((self.N_countries, self.N_countries))
        for i in range(self.N_countries):
            for j in range(self.N_countries):
                Z[i, j] += np.sum(
                    self.excel_dataframe.values[i * self.N_sectors:(i + 1) *
                                                self.N_sectors,
                                                j * self.N_sectors:(j + 1) *
                                                self.N_sectors])
        self.Z = Z
        return Z

    def calculate_country_x(self):
        """Calculate the country-aggregated x vector from the Input Data - Equation (2)"""
        x = np.zeros(self.N_countries)
        for i in range(self.N_countries):
            x[i] += np.sum(
                self.excel_dataframe.values[-1][i * self.N_sectors:(i + 1) *
                                                self.N_sectors])
        self.x = x
        return x

    def calculate_country_f(self):
        """Calculate the country-aggregated f vector from the Input Data - Equation (1)"""
        f = np.zeros(self.N_countries)
        for i in range(self.N_countries):
            tmp = self.excel_dataframe.values[i * self.N_sectors:(i + 1) *
                                              self.N_sectors,
                                              self.N_countries *
                                              self.N_sectors:-1]
            f[i] = np.sum(tmp[tmp > 0])
        self.f = f
        return f

    def calculate_beta(self):
        """Calculate the country-aggregated beta vector as a normalized f vector - Equation (3)"""
        beta = np.abs(self.f) / np.sum(np.abs(self.f))
        self.beta = beta
        return beta

    def calculate_eps(self,
                      TFP_file=None,
                      standardize=True,
                      normalize=True,
                      log=True):
        """
        Calculate the country-aggregated eps vector from the Total Factor Productivity data
        
        Input:
            TFP_file (pd.DataFrame): DataFrame containing information on Total Factor Productivity 
            standardize (bool): Whether to standardize the eps vector
            normalize (bool): Whether to normalize the eps vector
            log (bool): Whether to log-transform the eps vector. This means we are doing the transformations on eps instead of zeta 
         """

        if (TFP_file is None):
            eps = np.ones(self.N_countries)

        else:
            eps = np.ones(self.N_countries) * 100
            for delta_year in range(self.year - 1990):
                eps += TFP_file[1990 + delta_year].values

        if (log):
            eps = np.log(eps)

        if (standardize):
            scaler = StandardScaler(with_mean=False)
            eps = scaler.fit_transform(eps.reshape(-1, 1)).reshape(-1)

        if (normalize):
            eps = eps / np.sum(eps)

        self.eps = eps
        return eps

    def calculate_A(self):
        """Calculate the country-aggregated A matrix as the Z matrix normalized by the x vector - Equation (5)"""
        A = np.zeros(self.Z.shape)
        for i in range(A.shape[0]):
            for j in range(A.shape[1]):
                A[i, j] = self.Z[i, j] / self.x[j]
        self.A = A
        return A

    def calculate_L(self):
        """Calculate the country-aggregated inverse Leontief L matrix as the the inverse of I-A - Equation (7)"""
        L = np.linalg.inv(np.identity(self.A.shape[0]) - self.A)
        self.L = L
        return L

    def fixed_matrix_derivative(self, matrix, country_i, country_j):
        """
        Calculate the matrix derivative for a specific link of the network - Equation (8)
        
        Input:
            matrix (np.array(N,N)): Input matrix
            country_i (int): Index of input country
            country_j (int): Index of output country
         """
        dM_ij = np.zeros((self.N_countries, self.N_countries))
        for p in range(self.N_countries):
            for q in range(N_countries):
                dM_ij[p, q] = matrix[country_j, p] * matrix[q, country_i]
        return dM_ij

    def whole_matrix_derivative(self, matrix):
        """
        Calculate the matrix derivative for the entire matrix  - Equation (10)
        
        Input: 
            matrix (np.array(N,N)): Input matrix
         """
        dM = np.kron(matrix.T, matrix)
        return dM

    def calculate_L_matrix_derivative(self):
        """Calculate the whole matrix derivative of L  - Equation (10)"""
        dL = self.whole_matrix_derivative(self.L)
        self.dL = dL
        return dL

    def calculate_delta_A(self, scalar=0.01, change_type="additive"):
        """
        Calculate the change in A.
        
        Input:
            scalar (float): Amount of change. Depends on change_type.
            change_type (str): "multiplicative" or "additive" Type of change. 
                                If additive then each element of the change is equal to the scalar and equal for all links. 
                                If multiplicative then each element of the change is equal to the scalar multiplied with A.
         """
        if (change_type == "multiplicative"):
            delta_A = scalar * self.A
        elif (change_type == "additive"):
            delta_A = np.ones(self.A.shape) * scalar
        self.delta_A = delta_A
        return delta_A

    def calculate_delta_L(self):
        """Calculate the change in L as the derivative of L times the change in A: delta_L = dL * delta_A  - Equation (9)"""
        delta_L = self.dL * self.delta_A.reshape(-1)[:, None].T
        self.delta_L = delta_L
        return delta_L

    def calculate_full_P_matrix(self):
        """Calculate the full P matrix based on delta_L, eps and beta - Equation (12)"""
        beta_matrix = np.array([self.beta for j in range(self.N_countries)])
        beta_vec = beta_matrix.flatten()

        eps_matrix = np.array([self.eps for i in range(self.N_countries)])
        eps_vec = eps_matrix.T.flatten()

        p = np.zeros((self.N_countries**2, self.N_countries**2),
                     dtype=np.double)
        for u in range(self.N_countries):
            for v in range(self.N_countries):
                p[:, u * self.N_countries + v] = np.exp(
                    eps_vec * self.delta_L[:, u * self.N_countries + v] *
                    beta_vec)
        self.P = p
        return p

# Calculate and save CIOT classes

In [3]:
excel_data = dict()
CIOT_data = dict()

In [4]:
# Read WIOT tables for years 2000-2014
for year in np.arange(2000, 2015):
    excel_data[str(year)] = pd.read_excel('Data/WIOT/WIOT' + str(year) +
                                          '_Nov16_ROW.xlsb',
                                          engine='pyxlsb',
                                          skiprows=2,
                                          header=[0, 1, 2, 3],
                                          index_col=[0, 1, 2, 3])

In [5]:
# Read Total Factor Productivity
TFP_file_original = pd.read_excel('Data/TFP-NonWeighted.xlsx')

In [6]:
# Initialize CIOT classes and set corresponding country names
for year in np.arange(2000, 2015):
    CIOT_data[str(year)] = CIOT(excel_data[str(year)], year)
    CIOT_data[str(year)].get_country_names()

In [7]:
output_dir = "CIOT-Final-Additive/"

# Calculate all vectors and matrices and save for further use
for year in CIOT_data.keys():
    start = time.time()
    print(year)
    print("Calculating Z matrix...")
    CIOT_data[year].calculate_country_Z_matrix()
    print("Calculating x...")
    CIOT_data[year].calculate_country_x()
    print("Calculating f...")
    CIOT_data[year].calculate_country_f()
    print("Calculating beta...")
    CIOT_data[year].calculate_beta()
    print("Calculating eps...")
    CIOT_data[year].calculate_eps(TFP_file_original,
                                  standardize=True,
                                  normalize=True)

    print("Calculating A matrix...")
    CIOT_data[year].calculate_A()
    print("Calculating L matrix...")
    CIOT_data[year].calculate_L()
    print("Calculating L matrix derivative...")
    CIOT_data[year].calculate_L_matrix_derivative()
    print("Calculating delta A...")
    CIOT_data[year].calculate_delta_A(0.01, change_type="additive")
    print("Calculating delta L...")
    CIOT_data[year].calculate_delta_L()

    print("Calculating P...")
    CIOT_data[year].calculate_full_P_matrix()
    
    print("Saving Data...")
    with open(output_dir + 'CIOT_' + year + '.pkl', 'wb') as output:
        pickle.dump(CIOT_data[year], output, pickle.HIGHEST_PROTOCOL)
    end = time.time()
    print(end - start)

2000
Calculating Z matrix...
Calculating x...
Calculating f...
Calculating beta...
Calculating eps...
Calculating A matrix...
Calculating L matrix...
Calculating L matrix derivative...
Calculating delta A...
Calculating delta L...
Calculating P...
Saving Data...
32.18716096878052
2001
Calculating Z matrix...
Calculating x...
Calculating f...
Calculating beta...
Calculating eps...
Calculating A matrix...
Calculating L matrix...
Calculating L matrix derivative...
Calculating delta A...
Calculating delta L...
Calculating P...
Saving Data...
33.28763508796692
2002
Calculating Z matrix...
Calculating x...
Calculating f...
Calculating beta...
Calculating eps...
Calculating A matrix...
Calculating L matrix...
Calculating L matrix derivative...
Calculating delta A...
Calculating delta L...
Calculating P...
Saving Data...
32.285865783691406
2003
Calculating Z matrix...
Calculating x...
Calculating f...
Calculating beta...
Calculating eps...
Calculating A matrix...
Calculating L matrix...
Calcul